In [4]:
from joblib import dump, load
import json
import os, sys
import firebase_admin
from firebase_admin import firestore
from firebase_admin import credentials
import tqdm
import pandas as pd



In [2]:
cred = credentials.Certificate(r'bugbusters-c6a2c-firebase-adminsdk-5sm2b-50a8ebf152.json')
firebase_admin.initialize_app(cred)


In [3]:


#print current working directory
print(os.getcwd())



#load the model
model_incoming_bikes = load(r'random_forest_model_incoming.joblib')
model_outgoing_bikes = load(r'random_forest_model_outgoing.joblib')



c:\Users\ramir\Documents\Coding projects\Hackathon\BugBusters\data_training


In [14]:
df = pd.read_pickle(r'raw_data.pkl')
df.shape

(19484411, 10)

In [23]:
df_copy = df.copy()
# df_copy = df_copy.groupby(['STARTSTATIONARRONDISSEMENT', "ENDSTATIONARRONDISSEMENT"], as_index= False).count()

#keep only STARTSTATIONNAME == ville-marie
df_copy = df_copy[(df_copy['ENDSTATIONARRONDISSEMENT'] == 'Ville-Marie')]
df_copy.groupby(['ENDSTATIONARRONDISSEMENT', "ENDSTATIONNAME"], as_index= False).count()



,ENDSTATIONARRONDISSEMENT,ENDSTATIONNAME,STARTSTATIONNAME,STARTSTATIONARRONDISSEMENT,STARTSTATIONLATITUDE,STARTSTATIONLONGITUDE,ENDSTATIONLATITUDE,ENDSTATIONLONGITUDE,STARTTIMEMS,ENDTIMEMS
0,Ville-Marie,Alexandre-DeSève / Ontario,42195,42195,42195,42195,42230,42230,42230,42230
1,Ville-Marie,Alexandre-DeSève / Ste-Catherine,29839,29839,29839,29839,29864,29864,29864,29864
2,Ville-Marie,Atateken / du Square Amherst,58239,58239,58239,58239,58329,58329,58329,58329
3,Ville-Marie,Atwater / Sherbrooke,12584,12584,12584,12584,12602,12602,12602,12602
4,Ville-Marie,Aylmer / Ste-Catherine,24820,24820,24820,24820,24854,24854,24854,24854
...,...,...,...,...,...,...,...,...,...,...
232,Ville-Marie,du Havre / Hochelaga,2016,2016,2016,2016,2017,2017,2017,2017
233,Ville-Marie,du Havre / de Rouen,25371,25371,25371,25371,25395,25395,25395,25395
234,Ville-Marie,du President-Kennedy / Union,35636,35636,35636,35636,35701,35701,35701,35701
235,Ville-Marie,du Square Dorchester / Peel,310,310,310,310,310,310,310,310


In [24]:
#extract all elements in the STARTSTATIONNAME column
station_names = df_copy['STARTSTATIONNAME'].unique()
station_names = station_names.tolist()
# station_names

['Rielle / Wellington',
 'Clark / Ontario',
 'St-Antoine / de la Montagne',
 'de Maisonneuve / Greene',
 'Ste-Catherine / St-Laurent',
 'Marquette / Gillford',
 'de Bleury / de Maisonneuve',
 'St-Hubert / du Mont-Royal',
 'Poupart / Ontario',
 'McTavish / Sherbrooke',
 'Ste-Catherine / Dezery',
 'Prince-Arthur / St-Urbain',
 'Dorchester / Peel',
 'Marquette / du Mont-Royal',
 'St-Hubert / René-Levesque',
 'St-André / Ontario',
 'Valois / Ste-Catherine',
 'Chomedey / de Maisonneuve',
 "de l'Hôtel-de-Ville / Ste-Catherine",
 'Montcalm / Ontario',
 'Ste-Catherine / Towers',
 'Lincoln / du Fort',
 'Mackay / Ste-Catherine',
 'Jeanne-Mance / René-Lévesque',
 'Bloomfield / Bernard',
 'des Érables / Dandurand',
 'McGill / Wellington',
 'Métro Papineau (Dorion / De Maisonneuve)',
 'Drummond / de Maisonneuve',
 'Métro Place-des-Arts (du Président-Kennedy / Jeanne-Mance)',
 'Victoria / de Maisonneuve',
 'St-André / Rachel',
 'Laval / du Mont-Royal',
 'Parc du Pélican (2e avenue / St-Joseph)',
 'M

In [28]:
df_copy2 = df.copy()
# df_copy = df_copy.groupby(['STARTSTATIONARRONDISSEMENT', "ENDSTATIONARRONDISSEMENT"], as_index= False).count()

#keep only STARTSTATIONNAME == ville-marie
df_copy2 = df_copy2[(df_copy2['ENDSTATIONNAME'].isin(station_names))]

In [29]:
df_copy2.shape

(18445701, 10)

In [ ]:


#load json files
with open('id_to_station.json', 'r') as fp:
    station_to_id = json.load(fp)


#add to a list
model_incoming_bikes_arr = []
i = 0 
for key in (station_to_id.keys()):
    print ("Progress: ", i, "/", len(station_to_id.keys()))
    if (station_to_id[key] in station_names):#truncated for simplicity
        for month in range(4,11):
            for day_of_week in range(0,7):
                for hour in range(0,24):
                    model_incoming_bikes_arr.append ({"station_id": key, "station_name": station_to_id[key],"month": month, "day_of_week": day_of_week, "hour": hour, "prediction": model_incoming_bikes.predict([[key, month, hour, day_of_week]])})

print(model_incoming_bikes)


In [ ]:
db = firestore.client()

for data in model_incoming_bikes_arr:
    db.collection("BIKES_IN").document(data['station_id']).set(data)

In [ ]:

#add to a list
model_outgoing_bikes_arr = []
i = 0 
for key in (station_to_id.keys()):
    print ("Progress: ", i, "/", len(station_to_id.keys()))
    for month in range(0,12):
        for day_of_week in range(0,7):
            for hour in range(0,24):
                model_outgoing_bikes_arr.append ({"station_id": key, "station_name": station_to_id[key],"month": month, "day_of_week": day_of_week, "hour": hour, "prediction": model_outgoing_bikes.predict([[key, month, hour, day_of_week]])})

print(model_outgoing_bikes_arr)


In [ ]:
db = firestore.client()

for data in model_outgoing_bikes_arr:
    db.collection("BIKES_OUT").document(data['station_id']).set(data)